<a href="https://colab.research.google.com/github/ricardoricrob76/codigocoleta/blob/master/DIAD_Tabelas_Iceberg_com_Carga_Incremental_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Aqui está um exemplo básico de código em Tabelas Iceberg com carga incremental do tipo CDC (Change Data Capture):

In [ ]:
-- Criação da tabela Iceberg
CREATE TABLE iceberg_table (
  id INT,
  name VARCHAR(100),
  age INT,
  last_updated TIMESTAMP
) PARTITIONED BY (date DATE) STORED AS ICEBERG;

-- Criação da tabela de staging para o CDC
CREATE TABLE staging_table (
  id INT,
  name VARCHAR(100),
  age INT,
  operation VARCHAR(10),
  updated_at TIMESTAMP
) STORED AS PARQUET;

-- Inserção de dados na tabela de staging
INSERT INTO staging_table (id, name, age, operation, updated_at)
VALUES
  (1, 'John', 30, 'INSERT', CURRENT_TIMESTAMP),
  (2, 'Jane', 25, 'INSERT', CURRENT_TIMESTAMP),
  (1, 'John Doe', 31, 'UPDATE', CURRENT_TIMESTAMP),
  (2, 'Jane Smith', 26, 'UPDATE', CURRENT_TIMESTAMP),
  (3, 'Bob', 40, 'INSERT', CURRENT_TIMESTAMP);

-- Carga incremental para a tabela Iceberg
INSERT INTO iceberg_table
SELECT
  id,
  name,
  age,
  updated_at
FROM staging_table
WHERE operation IN ('INSERT', 'UPDATE');

-- Atualização da coluna last_updated na tabela Iceberg
UPDATE iceberg_table
SET last_updated = (SELECT MAX(updated_at) FROM staging_table);

-- Limpeza da tabela de staging após a carga incremental
DELETE FROM staging_table;


Neste exemplo, temos duas tabelas: iceberg_table, que é a tabela Iceberg onde os dados são armazenados, e staging_table, que é a tabela de staging usada para capturar as mudanças incrementais.

Na tabela staging_table, os dados são inseridos com uma coluna adicional chamada operation, que indica se a linha é uma inserção ou uma atualização. A coluna updated_at armazena a data e hora em que a operação foi executada.

A carga incremental é realizada na tabela iceberg_table usando uma instrução INSERT INTO ... SELECT que seleciona apenas as linhas da tabela de staging com operações de inserção ou atualização. Após a carga, a coluna last_updated na tabela iceberg_table é atualizada com a data e hora mais recentes das operações de atualização.

Por fim, a tabela de staging é limpa usando a instrução DELETE FROM staging_table, pois os dados já foram carregados na tabela Iceberg.

Este é apenas um exemplo básico para ilustrar a ideia de carga incremental do tipo CDC em Tabelas Iceberg. Dependendo do seu ambiente e necessidades específicas, pode ser necessário fazer ajustes adicionais.